# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## I) Scrap of the Wiki page to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

Scrap of the postal code table from the Wiki page ""https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050""

**Observation: Use of one old page because the current wiki page has already the neighborhoods of same postal code grouping on one unique row.

In [1]:
import pandas as pd
import numpy as np
URL='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'
#This function searches for <table> elements and only for <tr> and <th> rows and <td> elements
#within each <tr> or <th> element in the table. <td> stands for "table data". 
scrapWiki = pd.read_html(URL, match='Postcode') #Filter the table with Postcode in the header
print('Data downloaded!')

Data downloaded!


Renaming of the columns with postal code and with the neighborhood and pass the empty string as numpy.nan

In [2]:
#Renaming of the columns
postalCode_df = scrapWiki[0]
postalCode_df=postalCode_df.rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'})
print(postalCode_df.columns)

#Replace empty string by np.nan to have True with the Pandas function isnull
postalCode_df = postalCode_df.replace('', np.nan)

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')


Only consideration of the row with a borough associated

In [3]:
postalCode_df=postalCode_df[(postalCode_df.Borough!='Not assigned') & (~postalCode_df.Borough.isnull())]
postalCode_df.reset_index(drop=True, inplace=True)

For the cells without neighborhood assigned, use the borough as neighborhood

In [4]:
for ind in postalCode_df[(postalCode_df.Neighborhood == 'Not assigned') | (postalCode_df.Neighborhood.isnull())].index:
    print('index {}: Neighborhood informed with {}'.format(ind, postalCode_df.loc[ind, "Borough"]))
    postalCode_df.loc[ind, "Neighborhood"] = postalCode_df.loc[ind, "Borough"]


Consultation of an exenple of postal code, M9V. with more that one neighborhood, so with various rows.

In [5]:
postalCode_df[postalCode_df.PostalCode=='M9V']

,PostalCode,Borough,Neighborhood
173,M9V,Etobicoke,Albion Gardens
174,M9V,Etobicoke,Beaumond Heights
175,M9V,Etobicoke,Humbergate
176,M9V,Etobicoke,Jamestown
177,M9V,Etobicoke,Mount Olive
178,M9V,Etobicoke,Silverstone
179,M9V,Etobicoke,South Steeles
180,M9V,Etobicoke,Thistletown


For the code postal/borough associated at more of one neighborhood and so present in various rows, creation of one unique row by code postal/borough compiling the diferent neighborhood one this unique row.

In [6]:
#Grouping of the neighborhood by the postaCode and borough, separating these by a comma
postalCode_df=postalCode_df.groupby(by=['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
#Consultation of an exenple of postal code, M9V. with more that one neighborhood
postalCode_df[postalCode_df.PostalCode=='M9V']


,PostalCode,Borough,Neighborhood
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Shape of the final cleaned data frame

In [7]:
#Shape of the final cleaned data frame
print('Shape of postalCode_df: '+str(postalCode_df.shape))

Shape of postalCode_df: (103, 3)


## II) Get of the latitude and the longitude coordinates of each neighborhood.

Because the Google API for geolocalisation is not more free, use Arcgis as provider

In [8]:
import geocoder # import geocoder

Loop on each postal code of Toronto

In [13]:
# initialization of the list which will conserve the coordenates of each postal code of Toronto
coordLatLng_lst = []
for ind in postalCode_df.iloc[0:].index:
    postalCode=postalCode_df.loc[ind, "PostalCode"]
    # initialization
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        #g = geocoder.google('{}, Toronto, Ontario'.format(postalCode))
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postalCode))
        lat_lng_coords = g.latlng
        
    coordLatLng_lst.append(lat_lng_coords)
    print("index {} - Coordenates of the postal code {}: {}".format(ind, postalCode, lat_lng_coords))

index 0 - Coordenates of the postal code M1B: [43.80862623100006, -79.18991284599997]
index 1 - Coordenates of the postal code M1C: [43.78577865700004, -79.15736763799998]
index 2 - Coordenates of the postal code M1E: [43.76580607300008, -79.18528434099994]
index 3 - Coordenates of the postal code M1G: [43.77154467100007, -79.21813521299998]
index 4 - Coordenates of the postal code M1H: [43.76879106300004, -79.23881306799996]
index 5 - Coordenates of the postal code M1J: [43.74420268600005, -79.22872456899995]
index 6 - Coordenates of the postal code M1K: [43.72688089800005, -79.26569363099998]
index 7 - Coordenates of the postal code M1L: [43.71334044300005, -79.28494163099998]
index 8 - Coordenates of the postal code M1M: [43.72353760300007, -79.22835287899994]
index 9 - Coordenates of the postal code M1N: [43.696447877000026, -79.26564232899995]
index 10 - Coordenates of the postal code M1P: [43.76131000000004, -79.26993999999996]
index 11 - Coordenates of the postal code M1R: [43.7

index 94 - Coordenates of the postal code M9B: [43.65002250300006, -79.55408903099999]
index 95 - Coordenates of the postal code M9C: [43.637813150000056, -79.57648363299995]
index 96 - Coordenates of the postal code M9L: [43.75938139500005, -79.55717381699998]
index 97 - Coordenates of the postal code M9M: [43.733682102000046, -79.54438077099996]
index 98 - Coordenates of the postal code M9N: [43.70549635400005, -79.52037009099996]
index 99 - Coordenates of the postal code M9P: [43.69629612800003, -79.53312611699994]
index 100 - Coordenates of the postal code M9R: [43.68688713700004, -79.56550730099997]
index 101 - Coordenates of the postal code M9V: [43.74405485200003, -79.58120294599996]
index 102 - Coordenates of the postal code M9W: [43.71161519300006, -79.58835079199997]


Adition of thes coordenates at the data frame with the new columns Latitude and Longitude

In [14]:
postalCode_df["Latitude"] = np.array(coordLatLng_lst)[:, 0]
postalCode_df["Longitude"] = np.array(coordLatLng_lst)[:, 1]
#Observation: The list is passed as array because the slicing in nested list is not working
#             to recover the individual latitude ans longitude of each postal code coordenates.

In [15]:
postalCode_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.808626,-79.189913
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813


## III) Exploration and clusterization of the neighborhoods in Toronto

### Map of Toronto City with all his neighborhoods

Creation of one map centred on Toronto City

In [16]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


Add marker with the different neighborhood of Toronto

In [18]:
# add markers to map
for lat, lng, borough, neighborhood in zip(postalCode_df['Latitude'], postalCode_df['Longitude'], postalCode_df['Borough'], postalCode_df['Neighborhood']):
    label = '{} of {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

### Define Foursquare Credentials and Version

In [100]:
CLIENT_ID = 'XXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
TOKEN_ID= 'XXXX'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('TOKEN_ID:' + TOKEN_ID)

Your credentails:
CLIENT_ID: XXXX
CLIENT_SECRET:XXXX
TOKEN_ID:XXXX


### Exploration of ALL the neighborhoods in Toronto

Creation of one function to recover all the venues, in a circle determined by the parameter radius, of neighborhoods, passing in parameters the name, latitude and longitude of these neighborhoods

In [20]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run of the above function on each neighborhood

In [21]:
toronto_venues = getNearbyVenues(names=postalCode_df['Neighborhood'],
                                   latitudes=postalCode_df['Latitude'],
                                   longitudes=postalCode_df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

Consultation of the results

In [22]:
print(toronto_venues.shape) #Size of the results
toronto_venues.head()       #Header of the results

(2257, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.808626,-79.189913,Upper Rouge Trail,43.809988,-79.186147,Trail
1,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Chick-N-Joy,43.768752,-79.187982,Fried Chicken Joint
2,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
3,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Swiss Chalet,43.768122,-79.190493,Restaurant
4,"Guildwood, Morningside, West Hill",43.765806,-79.185284,Shoppers Drug Mart,43.770282,-79.185012,Pharmacy


Count of venues by neighborhood

In [23]:
toronto_venues.groupby('Neighborhood').count() #Count of venus by neighborhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",12,12,12,12,12,12
"Alderwood, Long Branch",9,9,9,9,9,9
...,...,...,...,...,...,...
Willowdale West,6,6,6,6,6,6
Woburn,3,3,3,3,3,3
"Woodbine Gardens, Parkview Hill",14,14,14,14,14,14


How may unique categories of the returned venues

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 260 uniques categories.


### Analyze of each neighborhood

One hot enconding of the diferent values of the column "Venue Category"

In [29]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Because it exists venue named "Neighborhood", change of the name of the one-hot column corresponding to this venue, to avoid the confusion with the name neighborhood column

In [67]:
#Presence of a venu category named "Neighborhood"
print(toronto_venues['Venue Category'].unique()[toronto_venues['Venue Category'].unique() == 'Neighborhood'])
#Venues of this category
toronto_onehot[toronto_onehot.Neighborhood==1]

['Neighborhood']


,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
858,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
916,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1761,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
#Column one-hot "Neighborhood" renamed in "Neighborhood-cat"
toronto_onehot=toronto_onehot.rename(columns={'Neighborhood':'Neighborhood-cat'})

In [70]:
# add neighborhood column (name of the neighborhood and not the venus category) back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(2257, 261)


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Consultation of the venues of the Neighborhood 'Guildwood, Morningside, West Hill'

In [80]:
toronto_onehot[toronto_onehot.Neighborhood=='Guildwood, Morningside, West Hill']

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
1,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by neighborhood and frequency of each category

In [82]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped

(99, 261)


,Neighborhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.02,0.0,0.01,0.01,0.000000,0.01,...,0.0,0.0,0.01,0.0,0.000000,0.0,0.0,0.0,0.01,0.0
1,Agincourt,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.083333,0.0,0.0,0.0,0.00,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Willowdale West,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
95,Woburn,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
96,"Woodbine Gardens, Parkview Hill",0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0
97,Woodbine Heights,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.083333,0.00,...,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.00,0.0


The 5 categories by neighborhood with more venus

In [83]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:] #Not consideration of the row with the neighborhood name
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
              venue  freq
0       Coffee Shop  0.11
1              Café  0.06
2    Clothing Store  0.04
3        Restaurant  0.04
4  Sushi Restaurant  0.03


----Agincourt----
              venue  freq
0  Sushi Restaurant   0.2
1    Breakfast Spot   0.2
2   Badminton Court   0.2
3       Supermarket   0.2
4      Skating Rink   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                     venue  freq
0         Sushi Restaurant   0.5
1                 Pharmacy   0.5
2      Monument / Landmark   0.0
3  New American Restaurant   0.0
4         Neighborhood-cat   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0   Fried Chicken Joint  0.08
1                  Park  0.08
2            Beer Store  0.08
3  Caribbean Restaurant  0.08
4          Liquor Store  0.08


----Alderwood, Long Branch----
               venue  freq
0        C

4  Residential Building (Apartment / Condo)   0.2


----Hillcrest Village----
                                      venue  freq
0  Residential Building (Apartment / Condo)  0.33
1                                      Park  0.33
2                                   Dog Run  0.33
3                                    Museum  0.00
4                              Noodle House  0.00


----Humber Bay Shores, Mimico South, New Toronto----
           venue  freq
0    Yoga Studio  0.17
1  Grocery Store  0.17
2           Park  0.17
3   Skating Rink  0.17
4   Tennis Court  0.17


----Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea----
                        venue  freq
0                        Bank   0.2
1                        Park   0.2
2                Tennis Court   0.2
3              Baseball Field   0.2
4  Construction & Landscaping   0.2


----Humber Summit----
                        venue  freq
0  Const

Conservation in a data frame of the top 10 categories by neighborhood

In [85]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [86]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Restaurant,Clothing Store,Salad Place,Hotel,Sushi Restaurant,Gym,Deli / Bodega,Thai Restaurant
1,Agincourt,Breakfast Spot,Skating Rink,Badminton Court,Sushi Restaurant,Supermarket,Field,Fast Food Restaurant,Fish & Chips Shop,Farmers Market,Dog Run
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Sushi Restaurant,Pharmacy,Yoga Studio,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Pizza Place,Grocery Store,Japanese Restaurant,Beer Store,Fried Chicken Joint,Liquor Store,Caribbean Restaurant,Park,Video Store,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Sandwich Place,Pub,Athletics & Sports,Convenience Store,Pharmacy,Gas Station,Farm


### Cluster of the neighborhood

Run of the k-means with 5 clusters

In [87]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 3, 2, 2, 2])

New dataframe with the clusters and the top 10 venus categories

In [88]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [90]:
toronto_merged = postalCode_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.808626,-79.189913,3.0,Trail,Yoga Studio,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Distribution Center
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785779,-79.157368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284,2.0,Pizza Place,Coffee Shop,Restaurant,Fast Food Restaurant,Breakfast Spot,Intersection,Mexican Restaurant,Bus Line,Sports Bar,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.771545,-79.218135,2.0,Coffee Shop,Korean Restaurant,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Yoga Studio,Doctor's Office
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813,3.0,Trail,Construction & Landscaping,Lounge,Gaming Cafe,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant


Use of the cluster code 5 for the neiborhood without cluster because they haven't venues associated

In [91]:
toronto_merged[toronto_merged["Cluster Labels"].isnull()]

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785779,-79.157368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,M1X,Scarborough,Upper Rouge,43.834768,-79.204101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,M4J,East York,East Toronto,43.687046,-79.333890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.694460,-79.410280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
toronto_merged.loc[toronto_merged["Cluster Labels"].isnull(), "Cluster Labels"] = 5

Visualisation of the clusters in the Toronto map

In [93]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examination of each cluster

In [94]:
numCluster = 0
print("Cluster {}: ".format(numCluster))
toronto_merged.loc[toronto_merged['Cluster Labels'] == numCluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 0: 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,0.0,Park,Dog Run,Residential Building (Apartment / Condo),Yoga Studio,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Distribution Center
23,North York,0.0,Convenience Store,Bank,Park,Yoga Studio,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant
25,North York,0.0,Food & Drink Shop,Construction & Landscaping,Park,Falafel Restaurant,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Farm
79,North York,0.0,Park,Bakery,Yoga Studio,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm
90,Etobicoke,0.0,Park,Yoga Studio,Distribution Center,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
93,Etobicoke,0.0,Park,Skating Rink,Baseball Field,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Yoga Studio,Doctor's Office
97,North York,0.0,Playground,Park,Distribution Center,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [95]:
numCluster = 1
print("Cluster {}: ".format(numCluster))
toronto_merged.loc[toronto_merged['Cluster Labels'] == numCluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 1: 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,1.0,Home Service,Yoga Studio,Farm,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market


In [96]:
numCluster = 2
print("Cluster {}: ".format(numCluster))
toronto_merged.loc[toronto_merged['Cluster Labels'] == numCluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 2: 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,2.0,Pizza Place,Coffee Shop,Restaurant,Fast Food Restaurant,Breakfast Spot,Intersection,Mexican Restaurant,Bus Line,Sports Bar,Fried Chicken Joint
3,Scarborough,2.0,Coffee Shop,Korean Restaurant,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Yoga Studio,Doctor's Office
5,Scarborough,2.0,Indian Restaurant,Fast Food Restaurant,Big Box Store,Sandwich Place,Yoga Studio,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant
6,Scarborough,2.0,Playground,Coffee Shop,Department Store,Discount Store,Convenience Store,Cosmetics Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
7,Scarborough,2.0,Bus Line,Intersection,Bakery,Coffee Shop,Soccer Field,Bus Station,Metro Station,Cupcake Shop,Dance Studio,Fish Market
...,...,...,...,...,...,...,...,...,...,...,...,...
98,York,2.0,Breakfast Spot,Pharmacy,Deli / Bodega,Park,Gift Shop,Cosmetics Shop,Costume Shop,Fish Market,Fish & Chips Shop,Field
99,Etobicoke,2.0,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Chinese Restaurant,Intersection,Discount Store,Cuban Restaurant,Dog Run,Fish & Chips Shop
100,Etobicoke,2.0,Bank,Liquor Store,Shopping Mall,Coffee Shop,Sandwich Place,American Restaurant,Pizza Place,Pharmacy,Field,Fast Food Restaurant
101,Etobicoke,2.0,Pizza Place,Grocery Store,Japanese Restaurant,Beer Store,Fried Chicken Joint,Liquor Store,Caribbean Restaurant,Park,Video Store,Pharmacy


In [97]:
numCluster = 3
print("Cluster {}: ".format(numCluster))
toronto_merged.loc[toronto_merged['Cluster Labels'] == numCluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 3: 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Trail,Yoga Studio,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Distribution Center
4,Scarborough,3.0,Trail,Construction & Landscaping,Lounge,Gaming Cafe,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant
19,North York,3.0,Trail,Construction & Landscaping,Yoga Studio,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farm,Farmers Market


In [98]:
numCluster = 4
print("Cluster {}: ".format(numCluster))
toronto_merged.loc[toronto_merged['Cluster Labels'] == numCluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 4: 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,4.0,Auto Garage,Yoga Studio,Farm,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Doctor's Office


In [99]:
numCluster = 5 #Cluster positionned manually with the neighborhood without venues 
print("Cluster {}: ".format(numCluster))
toronto_merged.loc[toronto_merged['Cluster Labels'] == numCluster, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 5: 


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,Scarborough,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,East York,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,Central Toronto,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
